In [1]:
import pandas as pd
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

import aurum



In [2]:
AUDITAR_MODELO = os.listdir('VS07')[0]

In [3]:
TICKER = pd.Series(AUDITAR_MODELO).str.extract('- (.*.).sav').values[0][0]
TICKER_DATA = aurum.data.get_ohlcv(TICKER, TREINO=False)

In [50]:
class AuditarModelo:

    import warnings
    warnings.filterwarnings('ignore')
    

    def __init__(self, NOME_MODELO):
        self.NOME_MODELO = NOME_MODELO
        self.TICKER = pd.Series(NOME_MODELO).str.extract('- (.*.).sav').values[0][0]
        self.MODELO = self.load_model(self.NOME_MODELO)
        self.FEATURES_MODELO = self.MODELO.feature_names_in_

        self.TICKER_TEST_DATA = self.load_ticker_test_data(self.TICKER)
        self.MODEL_PREDICTIONS = aurum.model_training.make_predictions(self.TICKER)

        self.ACCURACY = self.calculate_model_score('ACCURACY')
        self.PRECISION = self.calculate_model_score('PRECISION')
        self.ADPATED_SHARPE_RATIO = self.calculate_adapted_sharpe()
        self.MODEL_CUMULATIVE_RETURN = self.calculate_model_return()
        self.IBOV_BNH_RETURN = self.calculate_ibov_bnh_return()

    def __repr__(self) -> str:
        return f'Auditar Modelo -> {self.NOME_MODELO}'

    def load_model(self, NOME_MODELO):
        import joblib
        
        return joblib.load(f'models/Silver - {TICKER}.sav')

    def load_ticker_test_data(self, TICKER):

        import aurum

        df = aurum.data.get_ohlcv(self.TICKER, TREINO=False)
        df = aurum.ft.technical_indicators(df)

        return df

    def calculate_model_score(self, score_name):

        import numpy as np
        from sklearn.metrics import accuracy_score, precision_score

        score_map = {'ACCURACY': accuracy_score, 'PRECISION': precision_score}

        y_pred = self.MODELO.predict(self.TICKER_TEST_DATA[self.FEATURES_MODELO])[:-5]
        y_test = np.ravel(self.TICKER_TEST_DATA.dropna(subset='LEAK_Retorno').Alvo)

        metric = score_map[score_name](y_pred=y_pred, y_true=y_test)

        return metric

    
    def calculate_adapted_sharpe(self):

        import aurum

        IBOV_DATA = aurum.data.get_ohlcv('BOVA11.SA', TREINO=False)

        IBOV_DATA['RETORNO_DIARIO'] = IBOV_DATA['Close'].pct_change()


        MODEL_RETURN = (self.MODEL_PREDICTIONS.iloc[-20:, :].RETORNO_MODELO.cumprod() - 1).iloc[-6]
        RSK_FREE_RETURN = ((1 + IBOV_DATA.iloc[-20:, :].RETORNO_DIARIO).cumprod() - 1).iloc[-6]
        EXCESS_RETURN = (self.MODEL_PREDICTIONS.RETORNO_MODELO - IBOV_DATA.RETORNO_DIARIO).dropna()


        ADAPTED_SHARPE = (MODEL_RETURN - RSK_FREE_RETURN)/EXCESS_RETURN.std()

        return ADAPTED_SHARPE

    def calculate_model_return(self):

        import aurum

        MODEL_RETURN = self.MODEL_PREDICTIONS.RETORNO_ACUMULADO_MODELO.dropna().to_list()[-1]

        return MODEL_RETURN

    def calculate_ibov_bnh_return(self):

        import aurum

        IBOV_DATA = aurum.data.get_ohlcv('BOVA11.SA', TREINO=False)

        IBOV_BNH_RETURN = 1 + (IBOV_DATA.loc[len(IBOV_DATA) - 1, 'Close'] - IBOV_DATA.loc[0, 'Close'])/IBOV_DATA.loc[0, 'Close']

        return IBOV_BNH_RETURN


    def plot_accuracy_over_time(self):

        import plotly.graph_objects as go
        import aurum

        df = self.MODEL_PREDICTIONS[['Date', 'Alvo', 'Predictions']]
        df['TruePositive'] = (df.Alvo == df.Predictions).astype(int)
        df['CumulativeTruePositive'] = df.TruePositive.cumsum()
        df['Acurácia'] = df.CumulativeTruePositive/(df.index + 1)

        # Seleciona-se após 10 dias para normalizar a curva
        df = df.iloc[10:, :]
        
        fig = go.Figure()

        fig.add_trace(
            go.Scatter(
                x=df.Date,
                y=df.Acurácia,
                hoverinfo=None,
                name='Estratégia Aurum',
                line={                      
                    'color':'#ecab18',
                    'shape':'spline',
                    'smoothing':0.3,
                    'width':3
                    }
            )
        )

        fig.update_layout(
            plot_bgcolor='#f1f1f1',
            paper_bgcolor='#f1f1f1',
        )


        fig.update_xaxes(
                showgrid=False,
                showticklabels=True,
                fixedrange=True,
                hoverformat='%d/%m/%Y'   
        )

        fig.update_yaxes(
                showgrid=True,
                showticklabels=True,
                tickformat='.0%',
                gridcolor='#CFCFCE',
                fixedrange=True,
                color='#2B2B2B',
                hoverformat='.0%',
                range = [-0.01, 1.01]    
        )

        fig.update_layout(
            title='Acurácia do Modelo ao longo do tempo'
        )
        
        return fig
        


In [48]:
modelo_metrics = AuditarModelo(AUDITAR_MODELO)

In [51]:
modelo_metrics.plot_accuracy_over_time()

In [9]:
df = modelo_metrics.MODEL_PREDICTI

In [18]:
import plotly.graph_objects as go

figure.add_trace(
        go.Scatter(
            x=df.Date,
            y=df.RETORNO_ACUMULADO_MODELO,
            hoverinfo=None,
            name='Estratégia Aurum',
            line={                      
                'color':'#ecab18',
                'shape':'spline',
                'smoothing':0.3,
                'width':3
            }
        )
    )

In [19]:
px.line((df.Alvo == df.Predictions).astype('int').cumsum()/((df.Alvo == df.Predictions).index + 1))